In [1]:
from transformers import pipeline
classifier = pipeline("sentiment-analysis")


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
2022-09-13 11:03:49.200211: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-09-13 11:03:49.200901: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [5]:
import torch
from transformers import pipeline

speech_recognizer = pipeline(
    "automatic-speech-recognition", model="facebook/wav2vec2-base-960h")



TFWav2Vec2ForCTC has backpropagation operations that are NOT supported on CPU. If you wish to train/fine-tine this model, you need a GPU or a TPU
2022-09-13 11:14:26.302510: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at xla_ops.cc:296 : UNIMPLEMENTED: Could not find compiler for platform METAL: NOT_FOUND: could not find registered compiler for platform METAL -- check target linkage


UnimplementedError: Exception encountered when calling layer "conv" (type TFWav2Vec2WeightNormConv1D).

Could not find compiler for platform METAL: NOT_FOUND: could not find registered compiler for platform METAL -- check target linkage [Op:__inference__jit_compiled_convolution_op_3774]

Call arguments received by layer "conv" (type TFWav2Vec2WeightNormConv1D):
  • inputs=tf.Tensor(shape=(1, 49, 768), dtype=float32)

In [2]:
from datasets import load_dataset, Audio,cast_column

dataset = cast_column("audio", Audio(
    sampling_rate=speech_recognizer.feature_extractor.sampling_rate))


FileNotFoundError: Couldn't find a dataset script at /Users/user/TENAC/week-1-4/Week-4/in-context-learning/notebooks/PolyAI/minds14p/minds14p.py or any data file in the same directory. Couldn't find 'PolyAI/minds14p' on the Hugging Face Hub either: FileNotFoundError: Dataset 'PolyAI/minds14p' doesn't exist on the Hub. If the repo is private, make sure you are authenticated with `use_auth_token=True` after logging in with `huggingface-cli login`.

In [ ]:
classifier("We are very happy to show you the 🤗 Transformers library.")
[{'label': 'POSITIVE', 'score': 0.9998}]

# http://snap.standford.edu/data/amazon/productGraph/category/Files/reviews_Cell_Phones_and_Accessories_5.json.gz


[{'label': 'POSITIVE', 'score': 0.9998}]

In [ ]:
results = classifier(
    ["We are very happy to show you the 🤗 Transformers library.", 
     "We hope you don't hate it.","Ethiopia is a great country"])
for result in results:
    print(f"label: {result['label']}, with score: {round(result['score'], 4)}")


label: POSITIVE, with score: 0.9998
label: NEGATIVE, with score: 0.5309
label: POSITIVE, with score: 0.9998


In [ ]:
import gensim 
import pandas as pd

In [ ]:
df = pd.read_json(
    '../data/reviews_Cell_Phones_and_Accessories_5.json', lines=True)
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


In [ ]:
df.reviewText[0]

"They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again"

In [ ]:
reviewText =  df.reviewText.apply(gensim.utils.simple_preprocess)
reviewText

0         [they, look, good, and, stick, good, just, don...
1         [these, stickers, work, like, the, review, say...
2         [these, are, awesome, and, make, my, phone, lo...
3         [item, arrived, in, great, time, and, was, in,...
4         [awesome, stays, on, and, looks, great, can, b...
                                ...                        
194434    [works, great, just, like, my, original, one, ...
194435    [great, product, great, packaging, high, quali...
194436    [this, is, great, cable, just, as, good, as, t...
194437    [really, like, it, becasue, it, works, well, w...
194438    [product, as, described, have, wasted, lot, of...
Name: reviewText, Length: 194439, dtype: object

In [ ]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2,
    workers=4
    )

In [ ]:
model.build_vocab(reviewText,progress_per=10000)


In [ ]:
model.epochs

5

In [ ]:
model.train(reviewText, total_examples=model.corpus_count,epochs=model.epochs)

(61505431, 83868975)

In [ ]:
model.save('../models/word2vec-amazon-cell-accessories-reviews-short.model')

In [ ]:
model.wv.most_similar('bad')

[('terrible', 0.6758965253829956),
 ('shabby', 0.6071705222129822),
 ('horrible', 0.6001275777816772),
 ('good', 0.5831167101860046),
 ('sad', 0.5334086418151855),
 ('awful', 0.5246843099594116),
 ('okay', 0.511488139629364),
 ('poor', 0.5048754811286926),
 ('lame', 0.5011241436004639),
 ('ok', 0.4965687394142151)]

In [ ]:
model.wv.similarity(w1='cheap',w2='inexpensive')

0.5415329

In [ ]:
# this ensures that the current MacOS version is at least 12.3+
print(torch.backends.mps.is_available())
# this ensures that the current current PyTorch installation was built with MPS activated.
print(torch.backends.mps.is_built()
